In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

In [ ]:
from context import Context

context = Context()

FEES setup for pairs

In [ ]:
from contracts.pair_contract import PairContract
from utils.contract_retrievers import retrieve_pair_by_address

pair_contract: PairContract
pair_contract = retrieve_pair_by_address("erd1qqqqqqqqqqqqqpgqph6g8569lnvpgd3x569hd6n6qse2aw0w0n4sms6nzv")    # operating pair
mex_contract = retrieve_pair_by_address("erd1qqqqqqqqqqqqqpgqa0fsfshnff4n76jhcye6k7uvd7qacsq42jpsp6shh2") # egldmex contract

In [ ]:
from contracts.pair_contract import PairContract
from utils.contract_retrievers import retrieve_pair_by_address

pair_contract: PairContract
pair_contract = retrieve_pair_by_address("erd1qqqqqqqqqqqqqpgq0e9pmlzr0nk5nkulzcmessttsjkzr4xf0n4sue4r8e")    # operating pair

In [ ]:
# whitelist in egldmex pair for swap no fees
mex_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, pair_contract.address)

In [ ]:
# set where to swap and what to do with the fees
pair_contract.add_trusted_swap_pair(context.deployer_account, context.network_provider.proxy,
                                    [
                                        mex_contract.address,
                                        mex_contract.firstToken,
                                        mex_contract.secondToken
                                    ])

In [ ]:
pair_contract.set_fees_percents(context.deployer_account, context.network_provider.proxy,
                                [300, 100])

In [ ]:
from contracts.router_contract import RouterContract
router_contract: RouterContract
router_contract = context.get_contracts(config.ROUTER_V2)[0]

pair_contract.set_fee_on_via_router(context.deployer_account, context.network_provider.proxy, router_contract, 
                                [
                                    config.ZERO_CONTRACT_ADDRESS,
                                    mex_contract.secondToken
                                ])

In [ ]:
from contracts.pair_contract import AddLiquidityEvent

event = AddLiquidityEvent(pair_contract.firstToken, 127791780000000000000, 1, pair_contract.secondToken, 5000000000000000000, 1)
pair_contract.add_liquidity(context.network_provider, context.deployer_account, event)

In [ ]:
pair_contract.resume(context.deployer_account, context.network_provider.proxy)